In [1]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from scipy.io import loadmat
import scipy
import h5py
#import hdf5storage
import csv


colors = [    
'orange','firebrick','lawngreen','dodgerblue','crimson','orchid','slateblue',
'darkgreen','darkorange','indianred','darkviolet','deepskyblue','greenyellow',
'peru','cadetblue','forestgreen','slategrey','lightsteelblue','rebeccapurple',
'darkmagenta','yellow','hotpink']
import glob2

Autosaving every 180 seconds


In [6]:
# 
def make_video(fname_h5,
              fname_video):
    
    from tqdm import trange
    
    colors = [
        (0,0,255),
        (255,0,0),
        (255,255,0),
        (0,128,0)
        ]

    # 
    names = ['female','male','pup1','pup2']
    clrs = ['blue','red','cyan','green']

    # convert data from h5 to npy
    hf = h5py.File(fname_h5, 'r')
    keys = hf.keys()
    group2 = hf.get('tracks')
    tracks = []
    for k in range(len(group2)):
        tracks.append(group2[k])

    tracks = np.array(tracks).transpose(3,0,2,1)
    np.save(fname_h5[:-3]+'.npy',tracks)
    print ("[n_frames, n_animals, n-features, xy,]", tracks.shape)
    
    # start loading tracks
    start = 0
    end = 100 #tracks.shape[0]
    
    # load and videos    
    video_name = fname_video
    fname_out = (video_name[:-4]+'_'+str(start)+"_"+str(end)+'.mp4')
    if os.path.exists(fname_out):
        print ("fname: exists", fname_out)
        return
    
    # video settings
    size_vid = np.array([1280,1024])
    dot_size = 4
    
    # load original vid
    original_vid = cv2.VideoCapture(video_name)
    fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
    video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0],size_vid[1]), True)
    original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

    # loop over frames
    for n in trange(start, end, 1):
    #for n in trange(start, 100, 1):
        ret, frame = original_vid.read()

        for i in range(tracks.shape[1]):
            color = colors[i] #(255, 0, 0)

            for f in range(tracks.shape[2]):
                x = tracks[n,i,f,0]
                y = tracks[n,i,f,1]

                if np.isnan(x) or np.isnan(y):
                    continue

                x = int(x)
                y = int(y)

                # plot cicrl
                center_coordinates = (x,y)
                radius = dot_size
                thickness = dot_size
                frame = cv2.circle(frame, center_coordinates, radius, color, thickness)

                # plot line
                if f<(tracks.shape[2]-1):
                    if np.isnan(tracks[n,i,f+1,0])==False:
                        start_point = (int(tracks[n,i,f+1,0]),
                                       int(tracks[n,i,f+1,1])) 

                        # End coordinate, here (250, 250) 
                        # represents the bottom right corner of image 
                        end_point = center_coordinates

                        # Line thickness of 9 px 
                        thickness = 3

                        # Using cv2.line() method 
                        # Draw a diagonal green line with thickness of 9 px 
                        frame = cv2.line(frame, 
                                         start_point, 
                                         end_point, 
                                         color, 
                                         thickness)

#                 frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
#                     matplotlib.colors.to_rgb(clrs[i]))*255.).astype('uint8')

        video_out.write(frame)

    video_out.release()
    original_vid.release()

In [8]:
# load h5 names and convert to .npy and make movie
fname_h5 = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020-3-16_02_54_39_170978_compressed/2020_3_16_02_54_39_170978_compressed.analysis.h5'
fname_video = '/media/cat/7e3d5af3-7d7b-424d-bdd5-eb995a4a0c62/dan/cohort1/march_16/2020-3-16_02_54_39_170978_compressed/2020_3_16_02_54_39_170978_compressed.avi'

# LOAD h5 file:
import parmap
if False:
    parmap.map(make_video, 
               filenames,
               pm_processes=12,
               pm_pbar=True)
    
else:
    make_video(fname_h5,
              fname_video)
        #make_video(filename)
    

  0%|          | 0/100 [00:00<?, ?it/s]

[n_frames, n_animals, n-features, xy,] (89987, 4, 14, 2)


100%|██████████| 100/100 [00:02<00:00, 41.30it/s]


In [3]:
###############################
clrs = ['red','blue','cyan','green']
names = ['female','male','pup1','pup2']
t=np.arange(tracks.shape[0])/25.
for k in range(tracks.shape[1]):
    temp = tracks[:,k]
    print (temp.shape)
    temp = np.nanmean(temp,axis=1)
    print (temp.shape)
    
    temp = np.sum(temp,axis=1)
    print (temp.shape)
    plt.plot(t,temp,c=clrs[k],label=names[k])
    
    print ('')
plt.legend(fontsize=20)
plt.xlabel("Time (sec)",fontsize=20)
plt.show()

(7500, 14, 2)
(7500, 2)
(7500,)

(7500, 14, 2)
(7500, 2)
(7500,)

(7500, 14, 2)
(7500, 2)
(7500,)

(7500, 14, 2)
(7500, 2)
(7500,)



<ipython-input-3-dab89e9db851>:8: RuntimeWarning: Mean of empty slice
  temp = np.nanmean(temp,axis=1)
<ipython-input-3-dab89e9db851>:8: RuntimeWarning: Mean of empty slice
  temp = np.nanmean(temp,axis=1)
<ipython-input-3-dab89e9db851>:8: RuntimeWarning: Mean of empty slice
  temp = np.nanmean(temp,axis=1)
<ipython-input-3-dab89e9db851>:8: RuntimeWarning: Mean of empty slice
  temp = np.nanmean(temp,axis=1)


In [4]:
# MAKE COMPARISON OF LABELED VS> UNLABELED VIDEOS
# start going through video and 

    

100%|██████████| 7500/7500 [02:16<00:00, 54.98it/s]


In [53]:
from pykalman import KalmanFilter
import numpy as np
import matplotlib.pyplot as plt
import time

measurements = np.asarray([(399,293),(403,299),(409,308),(416,315),(418,318),(420,323),(429,326),(423,328),(429,334),(431,337),(433,342),(434,352),(434,349),(433,350),(431,350),(430,349),(428,347),(427,345),(425,341),(429,338),(431,328),(410,313),(406,306),(402,299),(397,291),(391,294),(376,270),(372,272),(351,248),(336,244),(327,236),(307,220)])

initial_state_mean = [measurements[0, 0],
                      0,
                      measurements[0, 1],
                      0]

transition_matrix = [[1, 1, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 1],
                     [0, 0, 0, 1]]

observation_matrix = [[1, 0, 0, 0],
                      [0, 0, 1, 0]]

kf1 = KalmanFilter(transition_matrices = transition_matrix,
                  observation_matrices = observation_matrix,
                  initial_state_mean = initial_state_mean)

kf1 = kf1.em(measurements, n_iter=5)
(smoothed_state_means, smoothed_state_covariances) = kf1.smooth(measurements)

plt.figure(1)
times = range(measurements.shape[0])
plt.plot(times, measurements[:, 0], 'bo',
         times, measurements[:, 1], 'ro',
         times, smoothed_state_means[:, 0], 'b--',
         times, smoothed_state_means[:, 2], 'r--',)
plt.show()